### Setup

In [1]:
%run -i '../scripts/tf_test.py'

In [2]:
import tensorflow as tf
tf.enable_eager_execution()

from tensorflow.keras.preprocessing.image import ImageDataGenerator
tf.__version__

'1.13.1'

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17847603739389941670
]


## Start program

### Select a model

In [ ]:
'''
Model 
'''

img_size = [224,224]

mobile_net = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
mobile_net.trainable=False

model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(['0','1']), activation = 'softmax')
])

In [20]:
'''
Model 
'''

img_size = [299,299]

inception = tf.keras.applications.InceptionV3(input_shape=(299, 299, 3))
inception.trainable=False

model = tf.keras.Sequential([
  inception,
#   tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(['0','1']), activation = 'softmax')
])

In [ ]:
'''
Model 
'''
img_size = [299,299]

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(299, 299, 3)),
#     tf.keras.layers.Conv2D(32,2,input_shape=(224,224,3)),
#     tf.keras.layers.Dense(batch_size, activation=tf.nn.relu),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.sigmoid)
])

In [ ]:
'''
Model 
'''

img_size = [640,640]

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(640,640,3)),
    tf.keras.layers.Dense(batch_size, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(['0','1']), activation=tf.nn.sigmoid)
])

### Load Data

In [21]:
'''
Set Params

sample_size: 0-1
'''

sample_size = 0.5

batch_size = 32

In [22]:
'''
Load CSVs
'''

df = GetCSVs(sample_size)
print(df['Catenary'].value_counts())

img_paths,labels = GetPaths(df)

[Errno 2] File b'../data/output_images/China/CHN.csv' does not exist: b'../data/output_images/China/CHN.csv'
1    166
0    166
Name: Catenary, dtype: int64
332


In [23]:
'''
Split, shuffle, and batch images into train/test datasets
'''

train_image_label_ds, test_image_label_ds = SplitDataSet(img_paths, labels)

train_ds = ShuffleBatch(train_image_label_ds,len(img_paths),BATCH_SIZE = batch_size) 
test_ds = ShuffleBatch(test_image_label_ds,len(img_paths),BATCH_SIZE = batch_size)

(299, 299, 3)
(299, 299, 3)


### Compile, Fit, and Evaluate

In [24]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.Accuracy()])

In [25]:
steps_per_epoch = int(tf.ceil(len(img_paths)/batch_size).numpy())

model.fit(train_ds,steps_per_epoch=steps_per_epoch, epochs=5)

model.evaluate(test_ds,steps=steps_per_epoch)

Epoch 1/5
11/11 [==============================] - 66s 6s/step - loss: 0.6930 - accuracy: 0.0000e+00
Epoch 2/5
 3/11 [=======>......................] - ETA: 46s - loss: 0.6925 - accuracy: 0.0000e+00

KeyboardInterrupt: 